In [9]:
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

import string
import unidecode
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# import data

In [2]:
df = pd.read_csv("https://wagon-public-datasets.s3.amazonaws.com/Machine%20Learning%20Datasets/reviews.csv")
df.head()

/Users/yanyi/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (1,2) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,review_id,length_review,review_score,order_id,product_category_name,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,7bc2406110b926393aa56f80a40eba40,0,4,73fc7af87114b39712e6da79b0a377eb,esporte_lazer,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59,41dcb106f807e993532d446263290104,delivered,2018-01-11 15:30:49,2018-01-11 15:47:59,2018-01-12 21:57:22,2018-01-17 18:42:41,2018-02-02 00:00:00
1,80e641a11e56f04c1ad469d5645fdfde,0,5,a548910a1c6147796b98fdf73dbeba33,informatica_acessorios,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13,8a2e7ef9053dea531e4dc76bd6d853e6,delivered,2018-02-28 12:25:19,2018-02-28 12:48:39,2018-03-02 19:08:15,2018-03-09 23:17:20,2018-03-14 00:00:00
2,228ce5500dc1d8e020d8d1322874b6f0,0,5,f9e4b658b201a9f2ecdecbb34bed034b,informatica_acessorios,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24,e226dfed6544df5b7b87a48208690feb,delivered,2018-02-03 09:56:22,2018-02-03 10:33:41,2018-02-06 16:18:28,2018-02-16 17:28:48,2018-03-09 00:00:00
3,e64fb393e7b32834bb789ff8bb30750e,37,5,658677c97b385a9be170737859d3511b,ferramentas_jardim,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06,de6dff97e5f1ba84a3cd9a3bc97df5f6,delivered,2017-04-09 17:41:13,2017-04-09 17:55:19,2017-04-10 14:24:47,2017-04-20 09:08:35,2017-05-10 00:00:00
4,f7c4243c7fe1938f181bec41a392bdeb,100,5,8e6bfb81e283fa7e4f11123a3fb894f1,esporte_lazer,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53,5986b333ca0d44534a156a52a8e33a83,delivered,2018-02-10 10:59:03,2018-02-10 15:48:21,2018-02-15 19:36:14,2018-02-28 16:33:35,2018-03-09 00:00:00


In [17]:
from olist.data import Olist
data = Olist().get_data()

## filter out reviews according to data 

In [3]:
df = df[df['review_creation_date']<=df['order_estimated_delivery_date']]
df.head()

,review_id,length_review,review_score,order_id,product_category_name,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,7bc2406110b926393aa56f80a40eba40,0,4,73fc7af87114b39712e6da79b0a377eb,esporte_lazer,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59,41dcb106f807e993532d446263290104,delivered,2018-01-11 15:30:49,2018-01-11 15:47:59,2018-01-12 21:57:22,2018-01-17 18:42:41,2018-02-02 00:00:00
1,80e641a11e56f04c1ad469d5645fdfde,0,5,a548910a1c6147796b98fdf73dbeba33,informatica_acessorios,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13,8a2e7ef9053dea531e4dc76bd6d853e6,delivered,2018-02-28 12:25:19,2018-02-28 12:48:39,2018-03-02 19:08:15,2018-03-09 23:17:20,2018-03-14 00:00:00
2,228ce5500dc1d8e020d8d1322874b6f0,0,5,f9e4b658b201a9f2ecdecbb34bed034b,informatica_acessorios,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24,e226dfed6544df5b7b87a48208690feb,delivered,2018-02-03 09:56:22,2018-02-03 10:33:41,2018-02-06 16:18:28,2018-02-16 17:28:48,2018-03-09 00:00:00
3,e64fb393e7b32834bb789ff8bb30750e,37,5,658677c97b385a9be170737859d3511b,ferramentas_jardim,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06,de6dff97e5f1ba84a3cd9a3bc97df5f6,delivered,2017-04-09 17:41:13,2017-04-09 17:55:19,2017-04-10 14:24:47,2017-04-20 09:08:35,2017-05-10 00:00:00
4,f7c4243c7fe1938f181bec41a392bdeb,100,5,8e6bfb81e283fa7e4f11123a3fb894f1,esporte_lazer,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53,5986b333ca0d44534a156a52a8e33a83,delivered,2018-02-10 10:59:03,2018-02-10 15:48:21,2018-02-15 19:36:14,2018-02-28 16:33:35,2018-03-09 00:00:00


## filter out columns  

In [4]:
df = df[['order_id','product_category_name','review_comment_title','review_comment_message','review_score']]
df.head()

,order_id,product_category_name,review_comment_title,review_comment_message,review_score
0,73fc7af87114b39712e6da79b0a377eb,esporte_lazer,NaN,NaN,4
1,a548910a1c6147796b98fdf73dbeba33,informatica_acessorios,NaN,NaN,5
2,f9e4b658b201a9f2ecdecbb34bed034b,informatica_acessorios,NaN,NaN,5
3,658677c97b385a9be170737859d3511b,ferramentas_jardim,NaN,Recebi bem antes do prazo estipulado.,5
4,8e6bfb81e283fa7e4f11123a3fb894f1,esporte_lazer,NaN,Parabéns lojas lannister adorei comprar pela I...,5


## dropna and fillna

In [5]:
df.dropna(subset=['review_comment_title','review_comment_message'],inplace=True)

In [7]:
df['title_comment'] = df['review_comment_title'].fillna('')+' '+df['review_comment_message'].fillna('')

In [8]:
df.head()

,order_id,product_category_name,review_comment_title,review_comment_message,review_score,title_comment
9,b9bf720beb4ab3728760088589c62129,eletroportateis,recomendo,aparelho eficiente. no site a marca do aparelh...,4,recomendo aparelho eficiente. no site a marca ...
15,e51478e7e277a83743b6f9991dbfa3fb,informatica_acessorios,Super recomendo,"Vendedor confiável, produto ok e entrega antes...",5,"Super recomendo Vendedor confiável, produto ok..."
22,4fc44d78867142c627497b60a7e0228a,beleza_saude,Ótimo,Loja nota 10,5,Ótimo Loja nota 10
36,37e7875cdce5a9e5b3a692971f370151,esporte_lazer,Muito bom.,Recebi exatamente o que esperava. As demais en...,4,Muito bom. Recebi exatamente o que esperava. A...
38,e029f708df3cc108b3264558771605c6,pet_shop,Bom,"Recomendo ,",5,"Bom Recomendo ,"


# text pre-cleaning

In [10]:
def clean (text):
    
    for punctuation in string.punctuation:
        text = text.replace(punctuation, ' ') # Remove Punctuation
        
    lowercased = text.lower() # Lower Case
    
    unaccented_string = unidecode.unidecode(lowercased) # remove accents
    
    tokenized = word_tokenize(unaccented_string) # Tokenize
    
    words_only = [word for word in tokenized if word.isalpha()] # Remove numbers
    
    stop_words = set(stopwords.words('portuguese')) # Make stopword list
    
    without_stopwords = [word for word in words_only if not word in stop_words] # Remove Stop Words
    
    return " ".join(without_stopwords)

df['clean_text'] = df['title_comment'].apply(clean)

df.head()

,order_id,product_category_name,review_comment_title,review_comment_message,review_score,title_comment,clean_text
9,b9bf720beb4ab3728760088589c62129,eletroportateis,recomendo,aparelho eficiente. no site a marca do aparelh...,4,recomendo aparelho eficiente. no site a marca ...,recomendo aparelho eficiente site marca aparel...
15,e51478e7e277a83743b6f9991dbfa3fb,informatica_acessorios,Super recomendo,"Vendedor confiável, produto ok e entrega antes...",5,"Super recomendo Vendedor confiável, produto ok...",super recomendo vendedor confiavel produto ok ...
22,4fc44d78867142c627497b60a7e0228a,beleza_saude,Ótimo,Loja nota 10,5,Ótimo Loja nota 10,otimo loja nota
36,37e7875cdce5a9e5b3a692971f370151,esporte_lazer,Muito bom.,Recebi exatamente o que esperava. As demais en...,4,Muito bom. Recebi exatamente o que esperava. A...,bom recebi exatamente esperava demais encomend...
38,e029f708df3cc108b3264558771605c6,pet_shop,Bom,"Recomendo ,",5,"Bom Recomendo ,",bom recomendo


# data processing

In [11]:
#turn review score into int

df['review_score'] = df['review_score'].astype(int)

In [12]:
# Groupby product category and aggregate mean, min, max review scores

product_performance = df.groupby('product_category_name').agg({'review_score': ['count','mean', 'min', 'max']}).sort_values([('review_score','mean')],ascending=False)
product_performance.head()

review_score                  
                                     count      mean min max
product_category_name                                       
fashion_roupa_infanto_juvenil            1  5.000000   5   5
tablets_impressao_imagem                 1  5.000000   5   5
cds_dvds_musicais                        1  5.000000   5   5
artigos_de_natal                        11  4.909091   4   5
fashion_calcados                        15  4.733333   4   5

In [13]:
# Keep categories that have more than 100 reviews

product_performance = product_performance[product_performance[('review_score','count')] >=100]
product_performance.tail()

review_score                  
                              count      mean min max
product_category_name                                
informatica_acessorios          554  3.859206   1   5
telefonia                       354  3.850282   1   5
relogios_presentes              801  3.845194   1   5
bebes                           248  3.786290   1   5
moveis_escritorio               117  3.376068   1   5

In [14]:
# Filter out category

relogios_presentes = df[df['product_category_name'].isin(['relogios_presentes'])]
relogios_presentes.head()

,order_id,product_category_name,review_comment_title,review_comment_message,review_score,title_comment,clean_text
61,8a9424899aac432d80d8e580932b5ee9,relogios_presentes,MT lindo,MT lindo,5,MT lindo MT lindo,mt lindo mt lindo
338,43050a51a257b17d51295e07650cb654,relogios_presentes,Lamenetável...,"Boa Noite\r\n\r\né lamentável, esta loja que t...",1,"Lamenetável... Boa Noite\r\n\r\né lamentável, ...",lamenetavel boa noite lamentavel loja tantas c...
566,0bb77e62d791c3e5b009815ac19825be,relogios_presentes,Bom,Muito bom mesmo!,5,Bom Muito bom mesmo!,bom bom
705,e87c331d21a3fb3085079a9b32f4a56c,relogios_presentes,10,"Gostei muito , o objeto veio o mesmo comprado,...",5,"10 Gostei muito , o objeto veio o mesmo compra...",gostei objeto veio comprado ja comprei outras ...
756,93aa1607392862dacfb4b54774480f1d,relogios_presentes,Regular,Loja em si é boa pena a americana nao da supor...,5,Regular Loja em si é boa pena a americana nao...,regular loja si boa pena americana nao suporte...


In [15]:
# Filter out bad reviews

relogios_presentes_bad_reviews = relogios_presentes[relogios_presentes['review_score'].isin([1])]
relogios_presentes_bad_reviews.head()

,order_id,product_category_name,review_comment_title,review_comment_message,review_score,title_comment,clean_text
338,43050a51a257b17d51295e07650cb654,relogios_presentes,Lamenetável...,"Boa Noite\r\n\r\né lamentável, esta loja que t...",1,"Lamenetável... Boa Noite\r\n\r\né lamentável, ...",lamenetavel boa noite lamentavel loja tantas c...
2859,836f9b5adb4fe06d8980fb71fa06b691,relogios_presentes,PÉSSIMO SERVIÇO,Já solicitei a troca porém até o momento nada....,1,PÉSSIMO SERVIÇO Já solicitei a troca porém at...,pessimo servico ja solicitei troca porem ate m...
3248,ae820ecd285ee7d0db7eac6f69049b36,relogios_presentes,Péssima loja,"Comprei um relogio, veio outro, mandei msgs, n...",1,"Péssima loja Comprei um relogio, veio outro, m...",pessima loja comprei relogio veio outro mandei...
3817,232ac1c81c6a84fd76f9653b248b5d92,relogios_presentes,Produto falsificado,O produto é falsificado. Não veio com manual e...,1,Produto falsificado O produto é falsificado. N...,produto falsificado produto falsificado nao ve...
4268,da0c01c648efe58143eae5e7c1a38ac0,relogios_presentes,NÃO entregue,Comprei 2 relógios mas só chegou 1. Já mandei ...,1,NÃO entregue Comprei 2 relógios mas só chegou ...,nao entregue comprei relogios so chegou ja man...


# text preprocessing

In [18]:
# vectorize

vec = TfidfVectorizer(ngram_range = (2,2), min_df=0.01, max_df = 0.05).fit(relogios_presentes_bad_reviews.clean_text)

vectors = vec.transform(relogios_presentes_bad_reviews.clean_text) # Transform text to vectors

sum_tfidf = vectors.sum(axis=0) # Sum of tfidf weighting by word

tfidf_list = [(word, sum_tfidf[0, idx]) for word, idx in vec.vocabulary_.items()]  # Get the word and associated weight

sorted_tfidf_list =sorted(tfidf_list, key = lambda x: x[1], reverse=True)  # Sort

sorted_tfidf_list

[('produto entregue', 4.823299861915626),
 ('ainda nao', 3.8500301309682508),
 ('propaganda enganosa', 3.6974097851505032),
 ('produto defeito', 3.4523848836568005),
 ('nao original', 3.2173656380879407),
 ('relogio veio', 3.2145017859559015),
 ('recebi relogio', 2.977055535926564),
 ('nao funciona', 2.8236647422272614),
 ('outro modelo', 2.696081824765045),
 ('entregue nao', 2.639529201040295),
 ('nao gostei', 2.632932643390745),
 ('relogio nao', 2.6108160304768457),
 ('relogio falso', 2.5386933138724315),
 ('nao veio', 2.435464016235141),
 ('errado nao', 2.418869502203795),
 ('troca produto', 2.411741318588243),
 ('produto falsificado', 2.3316511085499565),
 ('nao vale', 2.2588458012151515),
 ('veio defeito', 2.2537904159474897),
 ('diferente comprei', 2.005157255413781),
 ('insatisfacao negativo', 2.0),
 ('produto falso', 2.0),
 ('nao consigo', 1.9982255344127589),
 ('solicitei devolucao', 1.9838122896876063),
 ('quero devolver', 1.8984986276602025),
 ('agora nao', 1.892892708680781

# check worst sellers

In [19]:
sellers = data['order_items'].merge(relogios_presentes_bad_reviews)

In [20]:
# Filter out reviews with words associated with conterfeit watches

bad_sellers = sellers[sellers['clean_text'].str.contains("nao original|falso|outro modelo|produto falsificado|produto diferente")]
bad_sellers

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,product_category_name,review_comment_title,review_comment_message,review_score,title_comment,clean_text
4,0987363c91de454d42d084b84a691f99,1,d285360f29ac7fd97640bf0baef03de0,2eb70248d66e0e3ef83659f71b244378,2018-07-25 15:06:12,155.97,8.35,relogios_presentes,Relógio falso,"Relógio falso, me admiro com uma loja grande c...",1,"Relógio falso Relógio falso, me admiro com uma...",relogio falso relogio falso admiro loja grande...
7,0c23c36327d99027c61ca570921b0fb7,1,a92930c327948861c015c919a0bcb4a8,6560211a19b47992c3666cc44a7e94c0,2018-07-26 03:50:17,78.00,18.65,relogios_presentes,Réplica,"Infelizmente o relógio não é original, achei u...",1,"Réplica Infelizmente o relógio não é original,...",replica infelizmente relogio nao original ache...
13,232ac1c81c6a84fd76f9653b248b5d92,1,6bca83dbf6e081bccb2b23188577ad36,4869f7a5dfa277a7dca6462dcf3b52b2,2018-07-31 17:55:16,288.00,20.12,relogios_presentes,Produto falsificado,O produto é falsificado. Não veio com manual e...,1,Produto falsificado O produto é falsificado. N...,produto falsificado produto falsificado nao ve...
15,2ca33108764fd34547c251d1a4ad73ef,1,c0eeedfd383f32ae9524a15a8898b129,c60b801f2d52c7f7f91de00870882a75,2018-05-08 22:30:42,210.00,16.35,relogios_presentes,Produto errado e falso,Me foi entregue o modelo errado e não e origin...,1,Produto errado e falso Me foi entregue o model...,produto errado falso entregue modelo errado na...
20,31c9a3f10eb368b62e102608adbec724,1,64b4f32393cbf55be791078e7a2adca0,612170e34b97004b3ba37eae81836b4c,2018-07-31 11:41:16,139.90,19.08,relogios_presentes,Produto não original,Produto não informado como paralelo/réplica,1,Produto não original Produto não informado co...,produto nao original produto nao informado par...
26,3ec53ff8dd23b7e8a90cfb4082cf3f85,1,e0d64dcfaa3b6db5c54ca298ae101d05,2eb70248d66e0e3ef83659f71b244378,2018-06-26 21:30:52,124.90,16.84,relogios_presentes,Péssimo atendimento,O produto demorou aproximadamente 50 dias para...,1,Péssimo atendimento O produto demorou aproxima...,pessimo atendimento produto demorou aproximada...
28,3f7e2597afafe07f94a15e7f5da01135,1,2ffdf10e724b958c0f7ea69e97d32f64,4869f7a5dfa277a7dca6462dcf3b52b2,2018-05-08 20:53:31,179.90,19.14,relogios_presentes,Produto diferente,"Olá, a descrição do produto não corresponde co...",1,"Produto diferente Olá, a descrição do produto ...",produto diferente ola descricao produto nao co...
39,4d280f7e02765cbee0a2ce0359dafb2e,1,b84520a57891e7a8ae2c68741dcc7146,6560211a19b47992c3666cc44a7e94c0,2018-07-03 15:29:48,45.00,7.58,relogios_presentes,Relógio veio danificado,Relógio veio danificado e não é original,1,Relógio veio danificado Relógio veio danifica...,relogio veio danificado relogio veio danificad...
65,74fedc6aa056c9359846751fe21a8676,1,d285360f29ac7fd97640bf0baef03de0,2eb70248d66e0e3ef83659f71b244378,2018-07-25 09:04:01,154.91,16.18,relogios_presentes,Produto falsificado,Comprei o relógio casio rose Gold e o que cheg...,1,Produto falsificado Comprei o relógio casio r...,produto falsificado comprei relogio casio rose...
80,860e7b15d9daf5ec2b4357fb6cb57f43,1,3225c54bb2785c33f49f50398fcdb88c,c60b801f2d52c7f7f91de00870882a75,2018-06-08 14:54:46,225.00,16.45,relogios_presentes,Produto FALSIFICADO,Produto barato vendido no camelo por 20 reais ...,1,Produto FALSIFICADO Produto barato vendido no ...,produto falsificado produto barato vendido cam...


In [21]:
# Groupby seller id 

bad_sellers.groupby('seller_id').agg({'seller_id': ['count']})

,seller_id
,count
seller_id,
2eb70248d66e0e3ef83659f71b244378,11
4869f7a5dfa277a7dca6462dcf3b52b2,2
58f1a6197ed863543e0136bdedb3fce2,1
612170e34b97004b3ba37eae81836b4c,1
6560211a19b47992c3666cc44a7e94c0,2
7d13fca15225358621be4086e1eb0964,2
c60b801f2d52c7f7f91de00870882a75,3
d921b68bf747894be13a97ae52b0f386,1


In [22]:
# Filter out the one seller with 11 counterfeit related reviews

bad_sellers[bad_sellers["seller_id"]== "2eb70248d66e0e3ef83659f71b244378"].sort_values(by='shipping_limit_date')

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,product_category_name,review_comment_title,review_comment_message,review_score,title_comment,clean_text
104,a6ee9db107e8fd0fc14d99be85503396,1,e0d64dcfaa3b6db5c54ca298ae101d05,2eb70248d66e0e3ef83659f71b244378,2018-05-29 23:31:54,124.90,7.74,relogios_presentes,Meu produto veio errado,Meu produto veio errado outro modelo fui troca...,1,Meu produto veio errado Meu produto veio errad...,produto veio errado produto veio errado outro ...
26,3ec53ff8dd23b7e8a90cfb4082cf3f85,1,e0d64dcfaa3b6db5c54ca298ae101d05,2eb70248d66e0e3ef83659f71b244378,2018-06-26 21:30:52,124.90,16.84,relogios_presentes,Péssimo atendimento,O produto demorou aproximadamente 50 dias para...,1,Péssimo atendimento O produto demorou aproxima...,pessimo atendimento produto demorou aproximada...
139,e2b405a4cccf1e66eb8d2b73a4bba700,1,e0d64dcfaa3b6db5c54ca298ae101d05,2eb70248d66e0e3ef83659f71b244378,2018-06-27 09:35:22,124.90,16.84,relogios_presentes,Outro modelo/original???,Recebi um modelo de relógio diferente do que c...,1,Outro modelo/original??? Recebi um modelo de r...,outro modelo original recebi modelo relogio di...
114,b706ae4c6c85939c824f9ce25ae11884,1,6f3b5b605d91b7439c5e3f5a8dffeea7,2eb70248d66e0e3ef83659f71b244378,2018-07-23 16:31:12,128.90,16.00,relogios_presentes,Produto não original,Produto não original.,1,Produto não original Produto não original.,produto nao original produto nao original
143,e8aa7100127932a65507b2cb5ffdd901,1,6f3b5b605d91b7439c5e3f5a8dffeea7,2eb70248d66e0e3ef83659f71b244378,2018-07-23 18:11:00,138.90,19.07,relogios_presentes,Produto danificado e fals,"A loja me enviou um produto falso, danificado ...",1,Produto danificado e fals A loja me enviou um ...,produto danificado fals loja enviou produto fa...
116,b9d42d3c4b0b357f92488c43be786520,1,d285360f29ac7fd97640bf0baef03de0,2eb70248d66e0e3ef83659f71b244378,2018-07-25 07:10:17,154.91,8.34,relogios_presentes,Relógio falso,O relógio recebido não condiz com a foto do an...,1,Relógio falso O relógio recebido não condiz co...,relogio falso relogio recebido nao condiz foto...
65,74fedc6aa056c9359846751fe21a8676,1,d285360f29ac7fd97640bf0baef03de0,2eb70248d66e0e3ef83659f71b244378,2018-07-25 09:04:01,154.91,16.18,relogios_presentes,Produto falsificado,Comprei o relógio casio rose Gold e o que cheg...,1,Produto falsificado Comprei o relógio casio r...,produto falsificado comprei relogio casio rose...
88,8ffb58ad396250f240a3969dfbbc586c,1,d285360f29ac7fd97640bf0baef03de0,2eb70248d66e0e3ef83659f71b244378,2018-07-25 13:55:17,155.97,8.35,relogios_presentes,Quero devolver o produto,"Produto não é original, e não era isso que tin...",1,Quero devolver o produto Produto não é origin...,quero devolver produto produto nao original na...
4,0987363c91de454d42d084b84a691f99,1,d285360f29ac7fd97640bf0baef03de0,2eb70248d66e0e3ef83659f71b244378,2018-07-25 15:06:12,155.97,8.35,relogios_presentes,Relógio falso,"Relógio falso, me admiro com uma loja grande c...",1,"Relógio falso Relógio falso, me admiro com uma...",relogio falso relogio falso admiro loja grande...
122,c5a4102c4979beb7f062d7ad384cdac0,1,d285360f29ac7fd97640bf0baef03de0,2eb70248d66e0e3ef83659f71b244378,2018-07-26 00:04:19,155.97,16.19,relogios_presentes,Produto falsificado,"O relógio é falsificado, bem diferente da foto...",1,"Produto falsificado O relógio é falsificado, b...",produto falsificado relogio falsificado bem di...


# build model

In [24]:
X = df['clean_text']
y = df['review_score']

In [25]:
# vectorize

vectorizer = TfidfVectorizer(min_df=0.05).fit(X)
data_vectorized = vectorizer.transform(X)

# LDA

lda_model = LatentDirichletAllocation(n_components=3).fit(data_vectorized)

# print keywords

In [27]:
def print_topics(model, vectorizer):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        l = [(vectorizer.get_feature_names()[i], topic[i])
                        for i in topic.argsort()[:-10 - 1:-1]]
        print(l)
        

print_topics(lda_model, vectorizer)

Topic 0:
[('recomendo', 1117.116920856195), ('entrega', 627.8060976731173), ('super', 579.4535978319402), ('produto', 491.38154817757595), ('veio', 440.7824367499719), ('rapida', 312.7325260713817), ('loja', 307.7228827111482), ('prazo', 24.98355990357664), ('chegou', 7.248825360646958), ('entregue', 3.2257551499468047)]
Topic 1:
[('bom', 1104.061255554949), ('prazo', 767.749584717166), ('antes', 629.4919220965136), ('chegou', 603.3639400971886), ('produto', 439.2559528100383), ('tudo', 433.3027400035676), ('qualidade', 412.0143148796648), ('bem', 404.63737925842804), ('comprei', 321.3037248660909), ('boa', 304.77922942129845)]
Topic 2:
[('nao', 950.922331846507), ('otimo', 814.889298363413), ('produto', 620.6647301965818), ('excelente', 551.7872845815207), ('recebi', 509.49262404934535), ('gostei', 323.50145206056914), ('compra', 310.26589443581236), ('entregue', 156.9323408317192), ('entrega', 20.6414929251985), ('comprei', 10.662868123186948)]
